# pymysql 실습

- pip install pymysql을 설치후 하단 실습 실행

In [2]:
import pymysql

### 1. DB와 연결
- 필요한 정보 : id, pw, host, prt, db

In [3]:
# 127.0.0.1 = localhost : 문자로 localhost라고 치면, 컴터는 숫자 형태로 인식한다. 둘다 같음
# 336은 mysql의 default 코드 번호
# user의 root는 Admin거라서 보통 개발자 계정을 쓰지 admin을 쓰지는 않음. (보안이슈)
conn = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='jay',
    password = '1234',
    db ='csm_db',
    charset = 'utf8'
)

### 2. DB cursor 생성
- 데이터 셔틀버스와 같음
- 하단 코드 설명 : cur라는 cursor 객체를 만들었고, cur라는 코드 아래 execute라는 함수가 또 내장되어 있는 것.

In [4]:
cur = conn.cursor()

In [5]:
cur.execute("drop table if exists items")

0

### 3. sql 명령 실행
* 대문자 쓰는 건 개발자들 사이에서 가독성을 위한 관행 같은 것
- CREATE TABLE IF NOT EXISTS라는 조건이 없으면 재실행 시 오류

In [6]:
# items 테이블 만들기
CREATE_SQL ="""
CREATE TABLE IF NOT EXISTS items(
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    code TEXT NOT NULL,
    name TEXT NOT NULL,
    price INTEGER NULL
)

"""
cur.execute(CREATE_SQL);

#### 데이터 입력 (Create)

In [7]:
#insert_sql이라는 함수를 새로 정의하여서,
#기존에 insert into [db]라는 명령어를 반복 실행 했던 것을 간단하게 입력값만 넣음으로서, 수행하도록 하는 방법
# VALUES(%s, %s, %s) < 여기서 %s 이거는 데이터가 저장될 위치
insert_sql = """
INSERT INTO items(code, name, price)
    VALUES(%s, %s, %s);
"""

cur.execute(insert_sql, ('A001','TV',1000000))

#근데 이렇게 실행해도 db에 보이지는 않는게 보임. 왜냐? 이 창 역시도 개인 인벤토리 상태(메모리에만 저장)이기 때문.
#실행까지는 했다면, 이것을 반영하는 것이 중요

1

In [8]:
#DB 변경사항 저장, 반영은 pymysql.connect의 단계에서 수행하는 것
conn.commit()

In [9]:
# 여러개의 record를 한꺼번에 입력/추가하기
datas = (
    ('A0002', '에어컨 30평형', 2000000),
    ('A0003', '최신형 스마트폰', 1000000),
    ('A0004', '최신형 노트북', 500000)
)
cur.executemany(insert_sql, datas)
# 결과값의 '3'은 record 3개가 들어왔어~라는 뜻

3

In [10]:
#DB 변경사항 저장, 반영은 pymysql.connect의 단계에서 수행하는 것
conn.commit()

#### 데이터 읽기 (Read)

In [11]:
# select문
select_sql1 = "select * from items"
cur.execute(select_sql1)

4

In [12]:
#cursor가 db와의 셔틀버스 역할을 한다면, 
# fetch는 db에서 조회(커리)한 결과를 확인하기 위해 fetch 수행
rows = cur.fetchall()

In [13]:
rows
#수행해보니 결과 확인 = 이중 튜플로 저장되어 있음

((1, 'A001', 'TV', 1000000),
 (2, 'A0002', '에어컨 30평형', 2000000),
 (3, 'A0003', '최신형 스마트폰', 1000000),
 (4, 'A0004', '최신형 노트북', 500000))

In [14]:
#하나씩 보기 위한 for문 처리
for idx, row in enumerate(rows):
    print(row)

(1, 'A001', 'TV', 1000000)
(2, 'A0002', '에어컨 30평형', 2000000)
(3, 'A0003', '최신형 스마트폰', 1000000)
(4, 'A0004', '최신형 노트북', 500000)


In [15]:
for idx, row in enumerate(rows):
    print(f"{idx+1} 번째 - {row[1:]}")

1 번째 - ('A001', 'TV', 1000000)
2 번째 - ('A0002', '에어컨 30평형', 2000000)
3 번째 - ('A0003', '최신형 스마트폰', 1000000)
4 번째 - ('A0004', '최신형 노트북', 500000)


In [16]:
#select문, 2개의 레코드
select_sql1 = "select * from items limit 3"
cur.execute(select_sql1)

3

In [17]:
rows = cur.fetchall()
rows

((1, 'A001', 'TV', 1000000),
 (2, 'A0002', '에어컨 30평형', 2000000),
 (3, 'A0003', '최신형 스마트폰', 1000000))

In [18]:
cur.fetchall()
# 다시 fetch만 단독 실행하면, 아무것도 출력 X
# fetchall은 한번 수행하면 fetch가 저장된 공간의 데이터는 비워진다.

()

- fetchall()의 목적 : 데이터베이스와 python 사이의 다리 역할
[데이터베이스] ----SQL 쿼리----> [데이터베이스 서버]<br>
                                      ↓ (결과 생성)<br>
                                   [결과 집합]<br>
                                      ↓<br>
[Python 프로그램] <--fetchall()--- [커서(cursor)]<br>

왜 필요한가?
* SQL 쿼리를 실행(execute())하는 것과 결과를 가져오는 것은 별도의 작업
* execute()를 실행하면 데이터베이스 서버가 결과를 만듬
* 하지만 python으로는 전송되지 않고, cursor가 결과의 첫번째 행을 가리키고 있음
* 실행 후 데이터베이스 서버의 상태 :<br>
┌─────────────────┐<br>
│  Result Set     │  ← execute() 실행 후 여기 생성됨<br>
│  ┌───┬───┬───┐ │<br>
│  │ 1 │ A │ X │ │<br>  
│  │ 2 │ B │ Y │ │<br>  
│  │ 3 │ C │ Z │ │<br>  
│  └───┴───┴───┘ │<br>
│       ↑         │<br>
│    커서 포인터   │<br>
└─────────────────┘<br>

이때 fetchall()을 동작시켜서, 
1. 데이터베이스 서버에 "모든 행 보내줘!" 요청
2. 서버가 네트워크를 통해 데이터 전송
3. Python 메모리에 리스트로 저장
4. 커서 포인터가 결과 끝으로 이동

굳이 execute()와 분리된 이유? 효율성과 유연성 때문!
- execute는 메모리를 많이 사용하지만, 필요한 만큼만 실행시키면 메모리 절약이 가능

요약
**단계 / 무슨 일? / 데이터**
* 위치execute() / SQL 실행 / 데이터베이스 서버 메모리
* fetchall() / 결과 전송 요청 / Python 프로그램 메모리


#### update

In [19]:
update_sql = """
UPDATE items
SET price = %s
WHERE code = %s
"""

# UPDATE문(U)
price = 1500000
code = 'A001'
cur.execute(update_sql, (price, code))

1

In [20]:
conn.commit()

In [21]:
# id가 2인 데이터 list에 price의 record를 2500000으로 변경
update_sql_price = """
UPDATE items
SET price = %s
WHERE id = %s
"""

cur.execute(update_sql_price,(2500000,2))

1

In [22]:
conn.commit()

#### 데이터 삭제 delete

In [23]:
delete_sql = """
DELETE FROM items
WHERE code = %s
"""

cur.execute(delete_sql,'A0003')

1

In [54]:
conn.commit()

### 4. commit, rollback 실행
- insert, update, delete sql을 실행했을 경우에는 반드시 실행해야 db에 반영됨

### 5. DB close()
- 데이터 처리를 위해 연결한 DB연결 자원을 반환함.

In [41]:
# 데이터 생성하기
# code A
insert_sql = """
INSERT INTO items(code, name, price)
    VALUES(%s, %s, %s)
"""

cur.execute(insert_sql, ('A006','스타일러',3500000))

cur.execute(insert_sql, ('A005','로봇청소기',1000000))

1

In [42]:
select_sql1 = "select * from items"
cur.execute(select_sql1)
rows = cur.fetchall()
rows

((1, 'A001', 'TV', 1500000),
 (2, 'A0002', '에어컨 30평형', 2500000),
 (4, 'A0004', '최신형 노트북', 500000),
 (7, 'A0005', '로봇청소기', 1000000),
 (8, 'A0006', '스타일러', 7000000),
 (9, 'A006', '스타일러', 3500000),
 (10, 'A005', '로봇청소기', 1000000),
 (11, 'A006', '스타일러', 3500000),
 (12, 'A005', '로봇청소기', 1000000),
 (13, 'A006', '스타일러', 3500000),
 (14, 'A005', '로봇청소기', 1000000),
 (15, 'A006', '스타일러', 3500000),
 (16, 'A005', '로봇청소기', 1000000))

In [45]:
conn.rollback()

In [55]:
select_sql1 = "select * from items"
cur.execute(select_sql1)
rows = cur.fetchall()
rows

((1, 'A001', 'TV', 1500000),
 (2, 'A0002', '에어컨 30평형', 2500000),
 (4, 'A0004', '최신형 노트북', 500000),
 (7, 'A0005', '로봇청소기', 1000000),
 (8, 'A0006', '스타일러', 7000000))